In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py

In [37]:
path = "./data/"
datanum = 26 ##number of rows on the abundance table, modify as needed
ndigits= len(str(datanum))
raynum = 4 ##number of rays used, modify as needed

for r in range(raynum):
    rowlist = []
    for i in range(datanum):
        m = i+1
        n_len = len(str(m))
        n_zeros = ndigits - n_len
        k = "0" * n_zeros + str(m)
        row_data = pd.read_csv(path+f"data_AbundanceRow{k}_C_IV.txt", delim_whitespace=True) ##read in data files
        row_work = row_data[row_data["lightray_index"]==r] ##filter to only ray1
        df = row_work[["vel_dispersion","interval_start","interval_end"]].reset_index().drop(columns="index") ##filter to only indexes and velocites
        rowlist.append(df)
        

    mx = -np.inf
    for ds in rowlist:
        row_mx = max(ds["interval_end"])
        if row_mx > mx:
            mx = row_mx

    super_clumps = np.zeros(int(mx))

    clmaps = []
    for ds in rowlist:
        ds_clump_loc = np.zeros(int(mx))
        for i in range(ds.shape[0]):
            ds_clump_loc[int(ds['interval_start'][i]):int(ds['interval_end'][i])] = 1
            super_clumps[int(ds['interval_start'][i]):int(ds['interval_end'][i])] = 1
        clmaps.append(ds_clump_loc)

    super_clumps = np.append(0,super_clumps)
    super_clumps = np.append(super_clumps,0)
    
    
    match = {} ##create dictionaries to store indexes of clumps that correspond to one another
    shorter = {}
    merge = {}
    lonely = {}

    maybe_lonely = {}

    rownum = 0

    for row in clmaps: 

        row = np.append(0,row) # adding an extra element to prevent booleans from failing
        row = np.append(row,0)
        rownum += 1

        row_st_cnt = 0
        row_st_ind = []

        row_en_cnt = 0
        row_en_ind = []

        row_match = []
        row_short = []
        row_merge = []



        for i in range(1,len(row)):

            if super_clumps[i-1]<super_clumps[i]:
                sup_st = i-1

            if row[i-1]<row[i]:
                row_st_cnt += 1
                row_st_ind.append(i-1)

            elif row[i-1]>row[i]:
                row_en_cnt += 1
                row_en_ind.append(i-1)

            if super_clumps[i-1]>super_clumps[i]:
                sup_en = i-1

                if (row_st_cnt == 1) & (row_en_cnt == 1):
                    if (row_st_ind[0] == sup_st) & (row_en_ind[0] == sup_en):
                        row_match.append([row_st_ind[0],row_en_ind[0]])

                    else:
                        row_short.append([row_st_ind[0],row_en_ind[0]])

                elif (row_st_cnt == 0) & (row_en_cnt == 0):

                    if str([sup_st,sup_en]) in maybe_lonely.keys():
                        maybe_lonely[str([sup_st,sup_en])] += 1

                    else:
                        maybe_lonely[str([sup_st,sup_en])] = 1

                else:

                    for j in range(len(row_st_ind)):
                        row_merge.append([row_st_ind[j],row_en_ind[j]]) 

                row_st_cnt = 0
                row_st_ind = []

                row_en_cnt = 0
                row_en_ind = []

            match[rownum] = row_match
            shorter[rownum] = row_short
            merge[rownum] = row_merge

    false_merge = {}

    for i in range(rownum):
        for cat in [shorter, match, merge]:
            true_spans = []

            for n in range(len(rowlist[i])):
                true_spans.append([rowlist[i]["interval_start"][n],rowlist[i]["interval_end"][n].astype(int)])

            false_clumps = []

            for span in cat[i+1]:        

                if span in true_spans:
                    continue

                else:
                    true_span_arr = np.asarray(true_spans)


                    sp_st = span[0]
                    sp_end = span[1]

                    n = np.where(sp_st == true_span_arr[:,0])[0]
                    p = 0

                    while sp_end >= true_span_arr[n+p,1]:

                        false_clumps.append([int(true_span_arr[n+p,0]),int(true_span_arr[n+p,1])])
                        p+=1

                        if (n+p)>= len(true_span_arr[:,1]):
                            break

#                     cat[i+1].remove(span)

#                     false_merge[i+1] = false_clumps
                    
                    
                if i+1 in false_merge.keys():
                    false_merge[i+1].append(false_clumps[0])
                    false_merge[i+1].append(false_clumps[1])
                else:
                    false_merge[i+1] = false_clumps
                cat[i+1].remove(span) ##modify og dictionaries to help later 
                    
                    
#     print(false_merge)

[[181, 208], [208, 251]]
[[181, 208], [208, 251], [881, 892], [892, 901]]
[[181, 208], [208, 251]]
[[181, 208], [208, 251], [881, 892], [892, 901]]
[[181, 208], [208, 251]]
[[181, 208], [208, 251], [881, 892], [892, 901]]
[[181, 208], [208, 251]]
[[181, 208], [208, 251], [881, 892], [892, 901]]
[[181, 208], [208, 251]]
[[181, 208], [208, 251], [881, 892], [892, 901]]
[[181, 208], [208, 251]]
[[181, 208], [208, 251], [881, 892], [892, 901]]
[[181, 208], [208, 251]]
[[181, 208], [208, 251], [881, 892], [892, 901]]
[[181, 208], [208, 251]]
[[181, 208], [208, 251], [881, 892], [892, 901]]
[[181, 208], [208, 251]]
[[181, 208], [208, 251], [881, 892], [892, 901]]
[[181, 208], [208, 251]]
[[181, 208], [208, 251], [881, 892], [892, 901]]
[[181, 208], [208, 251]]
[[181, 208], [208, 251], [881, 892], [892, 901]]
[[181, 208], [208, 251]]
[[181, 208], [208, 251], [881, 892], [892, 901]]
[[181, 208], [208, 251]]
[[181, 208], [208, 251], [881, 892], [892, 901]]
[[181, 208], [208, 251]]
[[181, 208], 

In [27]:
import pandas as pd
import numpy as np
import pickle

path = "./data/"
datanum = 26 ##number of rows on the abundance table, modify as needed
ndigits= len(str(datanum))
raynum = 4 ##number of rays used, modify as needed

for r in range(raynum):
    r = 1
    rowlist = []
    for i in range(datanum):
        m = i+1
        n_len = len(str(m))
        n_zeros = ndigits - n_len
        k = "0" * n_zeros + str(m)
        row_data = pd.read_csv(path+f"data_AbundanceRow{k}_C_IV.txt", delim_whitespace=True) ##read in data files
        row_work = row_data[row_data["lightray_index"]==r] ##filter to only ray1
        df = row_work[["interval_start","interval_end"]].reset_index().drop(columns="index") ##filter to only indexes
        rowlist.append(df)

    mx= -np.inf  ##find how long each array should be
    for ds in rowlist: #find the cell index of the furthest clump
        row_mx = max(ds["interval_end"])
        if row_mx>mx:
            mx=row_mx
    super_clumps = np.zeros(int(mx))
    clmaps = []
    for ds in rowlist: ##make masks for each row and form super_clumps
        ds_clump_loc = np.zeros(int(mx))
        for i in range(1, ds.shape[0]):
            ds_clump_loc[int(ds["interval_start"][i]):int(ds["interval_end"][i])] = 1
            super_clumps[int(ds["interval_start"][i]):int(ds["interval_end"][i])] = 1
        clmaps.append(ds_clump_loc)
        super_clumps=np.append(0, super_clumps)  ##make indexing work  
    super_clumps=np.append(super_clumps, 0)
    np.save(f'super_clumps_array_ray{r}', super_clumps)
                    
    match = {} ##create dictionaries to store indexes of clumps in the row that correspond to one another, keys will be row numbers and values will be indecies except for the lonlies
    short = {}
    merge = {}
    false_merge ={}
    lonely = {}
    maybe_lonely = {}
    rownum = 0

    for row in clmaps:  ##start by iterating over a whole row
        row = np.append(0,row) # adding an extra element to prevent booleans from failing
        row = np.append(row,0)
        rownum += 1 ##define which row we're working on
    ##make all the variables and lists necessary
        row_st_cnt = 0 ##count how many starts of row clumps there have been within a super clump
        row_st_ind = []  ##keep track of start location(s) of a row clump

        row_en_cnt = 0 ##how many ends of row clumps there have been within a super clump
        row_en_ind = [] ##keep track of end location(s) of a row clump
        row_match = []
        row_short = []
        row_merge = []
    
        for i in range(1,len(row)):
            if super_clumps[i-1]<super_clumps[i]: ##start of a super clump
                sup_st = i-1 ##keep track of start location of a super clump
            if row[i-1]<row[i]:  ##start of a clump in the row
                row_st_cnt += 1
                row_st_ind.append(i-1)
        
            elif row[i-1]>row[i]: ##end of a clump in row
                row_en_cnt += 1
                row_en_ind.append(i-1)
            
            if super_clumps[i-1]>super_clumps[i]: ##end of a super clump
                sup_en = i-1 ##keep track of the location of the end of a super clump
          
                if (row_st_cnt == 1) and (row_en_cnt == 1): ##check for if there is only one row clump in the super clump
    
                    if (row_st_ind[0] == sup_st) & (row_en_ind[0] == sup_en): ##if the starts and ends match, the clumps are identical
                        row_match.append([row_st_ind[0],row_en_ind[0]]) ##thus, start and end indecies appended to a list of them
                    else:
                        row_short.append([row_st_ind[0],row_en_ind[0]]) ##if not, then the row clump must be shorter and the start and end indecies are appended to the appropraite list

                elif (row_st_cnt == 0) & (row_en_cnt == 0): ##check if there is nothing in the row that matches the super clump
                
                    if str([sup_st,sup_en]) in maybe_lonely.keys(): ##check if we have already seen this superclump, if not make the entry in the dictionary
                        maybe_lonely[str([sup_st,sup_en])] += 1
                    else:
                        maybe_lonely[str([sup_st,sup_en])] = 1
                else: ##only other senario is there there was a merge
                    for j in range(len(row_st_ind)): ##organize the indecies to make the list in order
                        row_merge.append([row_st_ind[j],row_en_ind[j]]) 
                                 
                row_st_cnt = 0
                row_st_ind = []
                row_en_cnt = 0
                row_en_ind = []
                match[rownum] = row_match
                short[rownum] = row_short
                merge[rownum] = row_merge

    for h in range(rownum):
        for dic in [match, short, merge]: ##iterate over all the dictionaries
            true_spans = [] ##array of indecies where clumps really are
            
            for n in range(len(rowlist[h])):
                true_spans.append([rowlist[h]["interval_start"][n], rowlist[h]["interval_end"][n].astype(int)])
                
            false_clumps = []
            
            for span in dic[h+1]:

                if span in true_spans: ##pass by if it really is a clump
                    continue
                    
                else: ##some weird merge happened so we have to fix it
                    true_span_arr = np.array(true_spans)
                    sp_st = span[0]
                    sp_en = span[1]
                    n = int(np.where(sp_st == true_span_arr[:,0])[0])
                    p = 0
                    
                    while sp_en >= true_span_arr[n+p, 1]:
                        false_clumps.append(int(true_span_arr[n+p, 0]))
                        false_clumps.append(int(true_span_arr[n+p, 1])) ##make it so we have the start and end indecies
                        p += 1
                        
                        if (n+p) >= len(true_span_arr[:,1]):
                            break
                            
                if h+1 in false_merge.keys():
                    false_merge[h+1].append(false_clumps)
                    
                else:
                    false_merge[h+1] = [false_clumps]
                dic[h+1].remove(span) ##modify og dictionaries to help later
                


In [29]:
# print(maybe_lonely)
# print(shorter)
# print(match)
print(merge)
# print(false_merge[1])

{1: [[390, 425], [442, 785], [795, 873], [905, 920], [934, 937], [973, 986], [988, 999], [1000, 1011], [1102, 1136], [1149, 1162]], 2: [[390, 425], [442, 785], [795, 873], [905, 920], [934, 937], [973, 986], [988, 999], [1000, 1011], [1102, 1136], [1149, 1162]], 3: [[390, 425], [442, 785], [795, 873], [905, 920], [934, 937], [973, 986], [988, 999], [1000, 1011], [1102, 1136], [1149, 1162]], 4: [[390, 425], [442, 785], [795, 873], [905, 920], [934, 937], [973, 986], [988, 999], [1000, 1011], [1102, 1136], [1149, 1162]], 5: [[390, 425], [442, 785], [795, 873], [905, 920], [934, 937], [973, 986], [988, 999], [1000, 1011], [1102, 1136], [1149, 1162]], 6: [[390, 425], [442, 785], [795, 873], [905, 920], [934, 937], [973, 986], [988, 999], [1000, 1011], [1102, 1136], [1149, 1162]], 7: [[390, 425], [442, 785], [795, 873], [905, 920], [934, 937], [973, 986], [988, 999], [1000, 1011], [1102, 1136], [1149, 1162]], 8: [[390, 425], [442, 785], [795, 873], [905, 920], [934, 937], [973, 986], [988, 

In [15]:
# df1_st = np.asarray(df1_clumps["interval_start"])
# df2_st = np.asarray(df2_clumps["interval_start"])
# df1_en = np.asarray(df1_clumps["interval_end"])
# df2_en = np.asarray(df2_clumps["interval_end"])


match = {} ##create dictionaries to store indexes of clumps that correspond to one another
shorter = {}
merge = {}
lonely = {}

maybe_lonely = {}

rownum = 0

for row in clmaps: 

    row = np.append(0,row) # adding an extra element to prevent booleans from failing
    row = np.append(row,0)
    rownum += 1

    row_st_cnt = 0
    row_st_ind = []
    
    row_en_cnt = 0
    row_en_ind = []

    row_match = []
    row_short = []
    row_merge = []
    
 
    
    for i in range(1,len(row)):
        
        if super_clumps[i-1]<super_clumps[i]:
            sup_st = i-1
            
        if row[i-1]<row[i]:
            row_st_cnt += 1
            row_st_ind.append(i-1)
        
        elif row[i-1]>row[i]:
            row_en_cnt += 1
            row_en_ind.append(i-1)
            
        if super_clumps[i-1]>super_clumps[i]:
            sup_en = i-1
            
            if (row_st_cnt == 1) & (row_en_cnt == 1):
                if (row_st_ind[0] == sup_st) & (row_en_ind[0] == sup_en):
                    row_match.append([row_st_ind[0],row_en_ind[0]])
            
                else:
                    row_short.append([row_st_ind[0],row_en_ind[0]])
            
            elif (row_st_cnt == 0) & (row_en_cnt == 0):
                
                if str([sup_st,sup_en]) in maybe_lonely.keys():
                    maybe_lonely[str([sup_st,sup_en])] += 1
                                 
                else:
                    maybe_lonely[str([sup_st,sup_en])] = 1
            
            else:
            
                for j in range(len(row_st_ind)):
                    row_merge.append([row_st_ind[j],row_en_ind[j]]) 
                                 
            row_st_cnt = 0
            row_st_ind = []
            
            row_en_cnt = 0
            row_en_ind = []
        
        match[rownum] = row_match
        shorter[rownum] = row_short
        merge[rownum] = row_merge

In [16]:
print(maybe_lonely)
print(shorter)
print(match)
print(merge)

{'[1745, 1763]': 1}
{1: [[164, 174], [181, 251], [345, 358], [934, 937], [1102, 1136], [1149, 1162], [1197, 1206], [1550, 1562]], 2: []}
{1: [[390, 425], [1242, 1252], [1467, 1475]], 2: [[160, 174], [178, 251], [344, 359], [390, 425], [442, 929], [934, 940], [971, 1011], [1102, 1138], [1148, 1162], [1197, 1210], [1242, 1252], [1467, 1475], [1548, 1564], [1745, 1763]]}
{1: [[442, 785], [795, 873], [881, 901], [905, 920], [973, 986], [988, 999], [1000, 1011]], 2: []}


In [20]:
false_merge = {}

for i in range(rownum):
    for cat in [shorter, match, merge]:
        true_spans = []
        
        for n in range(len(rowlist[i])):
            true_spans.append([rowlist[i]["interval_start"][n],rowlist[i]["interval_end"][n].astype(int)])

        false_clumps = []

        for span in cat[i+1]:        

            if span in true_spans:
                continue

            else:
                true_span_arr = np.asarray(true_spans)
                
                
                sp_st = span[0]
                sp_end = span[1]

                n = np.where(sp_st == true_span_arr[:,0])[0]
                p = 0

                while sp_end >= true_span_arr[n+p,1]:

                    false_clumps.append([int(true_span_arr[n+p,0]),int(true_span_arr[n+p,1])])
                    p+=1

                    if (n+p)>= len(true_span_arr[:,1]):
                        break
                
                cat[i+1].remove(span)
                
                false_merge[i+1] = false_clumps

In [21]:
false_merge

{1: [[881, 892], [892, 901]], 2: [[442, 881], [881, 929]]}

In [22]:
print(maybe_lonely)
print(shorter)
print(match)
print(merge)

{'[1745, 1763]': 1}
{1: [[164, 174], [345, 358], [934, 937], [1102, 1136], [1149, 1162], [1197, 1206], [1550, 1562]], 2: []}
{1: [[390, 425], [1242, 1252], [1467, 1475]], 2: [[160, 174], [178, 251], [344, 359], [390, 425], [934, 940], [971, 1011], [1102, 1138], [1148, 1162], [1197, 1210], [1242, 1252], [1467, 1475], [1548, 1564], [1745, 1763]]}
{1: [[442, 785], [795, 873], [905, 920], [973, 986], [988, 999], [1000, 1011]], 2: []}


In [120]:
true_span_arr = np.asarray(true_spans)

true_span_arr[:,0]

array([ 160,  178,  344,  390,  442,  881,  934,  971, 1102, 1148, 1197,
       1242, 1467, 1548, 1745])